In [1]:
import numpy as np
import pandas as pd

from spacebench.algorithms import DAPSm
from spacebench import SpaceDataset, DatasetEvaluator

WARNING ⚠️ : this package contains data with synthetic outcomes!
No inferences about the source data collection can be made.
By using it, you agree to understand its limitations and purpose.
The sole objective of SpaCE is to support the development of new
spatial confounding methods.



In [3]:
df = pd.read_csv("space/examples/data/dapsm_toydata.csv", index_col=0)
X = df[[c for c in df.columns if c.startswith("X")]].values
A = df.Z.values
beta = np.ones(X.shape[1])
Y0 = X @ beta
Y1 = Y0 + 10
Y = Y0 * (1 - A) + Y1 * A
long = df.long.values
lat = df.lat.values
dataset = SpaceDataset(
    treatment=A,
    covariates=X,
    outcome=Y,
    counterfactuals=np.stack([Y0, Y1], axis=1),
    coordinates=np.stack([long, lat], axis=1),
    edges=None,
    treatment_values=[0, 1],
)


In [5]:
method = DAPSm(spatial_weight=0.1)
evaluator = DatasetEvaluator(dataset)
method.fit(dataset)
effects = method.eval(dataset)
errors = evaluator.eval(**effects)

print("Estimated ATE:\n", effects['ate'])
print("True ATE:\n", Y1.mean() - Y0.mean())
print("Naive ATE estimate:\n", Y[A == 1].mean() - Y[A == 0].mean())
print("Evaluator errros:", errors)


Estimated ATE:
 5.081947774369604
True ATE:
 10.0
Naive ATE estimate:
 2.21141735563144
Evaluator errros: {'ate_error': np.float64(-0.08525807568842865), 'ate': np.float64(0.08525807568842865), 'atc_error': np.float64(-0.08525807568842865), 'atc': np.float64(0.08525807568842865), 'att_error': np.float64(-0.08525807568842865), 'att': np.float64(0.08525807568842865), 'ite_curve': array([0.04934668, 0.06952586]), 'ite': np.float64(0.059436267812260794), 'erf_error': array([ 0.02856146, -0.05669662]), 'erf': np.float64(0.04262903784421415)}
